In [1]:
import numpy as np
import math
import plotly.offline as py
from plotly.graph_objs import *

py.offline.init_notebook_mode()

In [2]:
def f(x):
    return x[0]**2 + 18*x[1]**2 + 0.01*x[0]*x[1] + x[0] - x[1]

In [3]:
h = 0.001
eps = 0.00001

x = [5., 5.]
rate = 0.5

theta = 0.5
delta = 0.5

In [4]:
def gradient(f, x):
    I = np.eye(len(x))
    return np.array([(f(x + h * I_) - f(x - h * I_))/(2.0*h) for I_ in I])
     
def hesse_matrix(f, x):
    I = np.eye(len(x))
    return np.array([(gradient(f, x + h*I_) - gradient(f, x - h*I_))/(2.0*h) for I_ in I])
    
def next_vector_gradient(f, x, rate):
    grads = -gradient(f, x)                                             
    return x + rate*grads

def next_vector_newton(f, x, rate):
    summs = -np.dot(np.linalg.inv(hesse_matrix(f, x)), gradient(f, x))                                           
    return x + rate*summs
                                                     
def error(f, x, next_x):
    args_error = np.linalg.norm(np.array(x) - np.array(next_x))
    func_error = abs(f(next_x) - f(x))
    grad_error = np.linalg.norm(gradient(f, next_x))
    return args_error, func_error, grad_error

In [5]:
class Optimizer:
    
    def __init__(self, x=x, rate=rate, theta=theta, delta=delta):
        self.rate = rate
        self.theta = theta
        self.delta = delta
        self.seq_x = [x]
    
    def modifiedGradientMethod(self, f):
        iteration = 1
        while True:
            next_x = next_vector_gradient(f, self.seq_x[-1], self.rate)
            rate = self.rate
            while f(next_x) - f(self.seq_x[-1]) > -self.theta*rate*np.linalg.norm(gradient(f, self.seq_x[-1]))**2: 
                rate = rate*self.delta
                next_x = next_vector_gradient(f, self.seq_x[-1], rate)

            errors = error(f, self.seq_x[-1], next_x)
            self.seq_x.append(next_x)
            print('{iteration} | x={x} | f(x)={f} | rate={rate} | errors=[{seq_x}, {val_f}, {grad_f}]'.format(
                iteration=iteration, x=self.seq_x[-2], f=f(self.seq_x[-2]), rate=rate, seq_x=errors[0], val_f=errors[1], grad_f=errors[2]))
            if errors[0] < eps and errors[1] < eps and errors[2] < eps : 
                break

            iteration += 1
    
    def modifiedNewtonMethod(self, f):
        iteration = 1
        while True:
            next_x = next_vector_newton(f, self.seq_x[-1], self.rate)
            rate = self.rate
            while f(next_x) - f(self.seq_x[-1]) > -self.theta*rate*np.dot(gradient(f,self.seq_x[-1]), -np.dot(np.linalg.inv(hesse_matrix(f, self.seq_x[-1])), gradient(f, self.seq_x[-1]))): 
                rate = rate*self.delta
                next_x = next_vector_newton(f, self.seq_x[-1], rate)
                
            errors = error(f, self.seq_x[-1], next_x)
            self.seq_x.append(next_x)
            print('{iteration} | x={x} | f(x)={f} | rate={rate} | errors=[{seq_x}, {val_f}, {grad_f}]'.format(
                iteration=iteration, x=self.seq_x[-2], f=f(self.seq_x[-2]), rate=rate, seq_x=errors[0], val_f=errors[1], grad_f=errors[2]))
            if errors[0] < eps and errors[1] < eps and errors[2] < eps : 
                break

            iteration += 1
    
    def plotSurface3d(self, f):
        seq_x = np.array(self.seq_x)
        x, y = seq_x.T
        axis_x = np.arange(np.min(x) - 2, np.max(x) + 2, 0.05)
        axis_y = np.arange(np.min(y) - 2, np.max(y) + 2, 0.05)
            
        f_values = np.array([[f([_x, _y]) for _y in axis_y] for _x in axis_x])
        curve_values = np.array([f([_x, _y]) for _x, _y in zip(x, y)])
                  
        trace = Scatter3d(
            mode='lines+markers',
            x=x, y=y, z=curve_values,
            marker=dict(size=7, color=curve_values, colorscale='Viridis',),
            line=dict(color='black', width=4)
        )
    
        data = Data([Surface(x=axis_x, y=axis_y, z=f_values), trace])
    
        layout = dict(
            title='Surface f(x)',
            autosize=False,
            width=500,
            height=500,
            margin=dict(l=65, r=20, b=65, t=90)
        )

        figure = dict(data=data,layout=layout)
        py.iplot(figure,filename='3dsurface')
        
    def plotSurface2d(self, f):
        seq_x = np.concatenate(self.seq_x)
        axis_x = np.arange(np.min(seq_x) - 2, np.max(seq_x) + 2, 0.05)
        f_values = np.array([f([_x]) for _x in axis_x])
        curve_values = np.array([f([_x]) for _x in seq_x])
        
        trace = Scatter(
            mode='lines+markers',
            x=seq_x, y=curve_values,
            marker=dict(size=7, color=curve_values, colorscale='Viridis',),
            line=dict(color='black', width=4)
        )
        data = Data([Scatter(x=axis_x, y=f_values), Scatter(x=seq_x, y=curve_values)])
        
        layout = dict(
            title='Surface f(x)',
            autosize=False,
            width=500,
            height=500,
            margin=dict(l=65, r=20, b=65, t=90)
        )

        figure = dict(data=data,layout=layout)
        py.iplot(figure,filename='2dsurface')


In [6]:
optimizer1 = Optimizer()
optimizer1.modifiedNewtonMethod(f)

1 | x=[5.0, 5.0] | f(x)=475.25 | rate=0.5 | errors=[3.707193939437829, 356.63552107185785, 89.69532457620322]
2 | x=[2.24993021 2.51395835] | f(x)=118.61447892814216 | rate=0.5 | errors=[1.8535969753307602, 89.15888030883688, 44.847662290273014]
3 | x=[0.87489531 1.27093753] | f(x)=29.45559861930528 | rate=0.5 | errors=[0.926798485402324, 22.289720072111958, 22.42383114551339]
4 | x=[0.18737786 0.64942712] | f(x)=7.165878547193324 | rate=0.5 | errors=[0.4633992429617823, 5.572430018723434, 11.211915572785923]
5 | x=[-0.15638086  0.33867191] | f(x)=1.59344852846989 | rate=0.5 | errors=[0.23169962151693133, 1.3931075047190016, 5.605957786387114]
6 | x=[-0.32826022  0.18329431] | f(x)=0.20034102375088855 | rate=0.5 | errors=[0.11584981075161203, 0.34827687617565395, 2.8029788931961335]
7 | x=[-0.4141999   0.10560551] | f(x)=-0.14793585242476542 | rate=0.5 | errors=[0.05792490537549966, 0.08706921904403145, 1.401489446598733]
8 | x=[-0.45716974  0.06676111] | f(x)=-0.23500507146879687 | ra

In [7]:
optimizer1.plotSurface3d(f)